<a href="https://colab.research.google.com/github/SaqlainHussainShah/SQLi-Detection-using-Machine-Learning/blob/master/Embeddings_for_SQLI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import glob
import time
import pandas as pd
# from xml.dom import minidom
from nltk import ngrams
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import PorterStemmer
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
from tensorflow import keras

In [0]:
import pandas as pd
import os


In [0]:
df=pd.read_csv('./sqli.csv',encoding='utf-16')

In [30]:
df.head()

,Sentence,Label
0,NaN,1
1,""" or pg_sleep ( __TIME__ ) --",1
2,create user name identified by pass123 tempora...,1
3,%29,1
4,' AND 1 = utl_inaddr.get_host_address ( ( S...,1


In [0]:
df.dropna(inplace=True)

In [0]:
values = df.values

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [0]:
sentences = []
for key , value in enumerate(values[:][:][:][:][:]):
    sentences.append((value[0]))

In [0]:
labels = []
for key , value in enumerate(values[:][:][:][:][:]):
    labels.append((value[1]))

In [36]:
sentences[:10]

['" or pg_sleep ( __TIME__ ) --',
 'create user name identified by pass123 temporary tablespace temp default tablespace users; ',
 '%29',
 "' AND 1 = utl_inaddr.get_host_address  (  (  SELECT DISTINCT ( table_name )  FROM  ( SELECT DISTINCT ( table_name ) , ROWNUM AS LIMIT FROM sys.all_tables )  WHERE LIMIT = 5  )  )   AND 'i' = 'i",
 " select * from users where id = '1' or @ @1 = 1 union select 1,version (  )  -- 1'",
 ' select * from users where id = 1 or 1#" (  union select 1,version (  )  -- 1',
 "' select name from syscolumns where id  =   ( select id from sysobjects where name  =  tablename' ) --",
 ' select * from users where id = 1 +$+ or 1 = 1 -- 1',
 '1; ( load_file ( char ( 47,101,116,99,47,112,97,115,115,119,100  )  )   ) ,1,1,1;',
 " select * from users where id = '1' or ||/1 = 1 union select 1,version (  )  -- 1'"]

In [0]:
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(oov_token=oov_tok, lower=False,split=' ')

# tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)

In [0]:
total_words = len(tokenizer.word_index) + 1

In [39]:
total_words

43941

In [0]:
import numpy as np

In [0]:
input_sequences = []
for line in sentences:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
    input_sequences.append(n_gram_sequence)

In [42]:
len(sentences)

33757

In [43]:
len(input_sequences)

33757

In [0]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [0]:
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [0]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

In [47]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len)) # Your Embedding Layer)
model.add(Bidirectional(LSTM(150, return_sequences=True)))# An LSTM Layer)
# model.add(Dropout(0.2)) # A dropout layer)
model.add(LSTM(100))# Another LSTM Layer)
model.add(Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01)))# A Dense Layer including regularizers)
model.add(Dense(1,activation='sigmoid'))# A Dense Layer)
# Pick an optimizer
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])# Pick a loss function and an optimizer)
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 544, 100)          4394100   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 544, 300)          301200    
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dense_2 (Dense)              (None, 512)               51712     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 513       
Total params: 4,907,925
Trainable params: 4,907,925
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
input_sequences = np.array(input_sequences,dtype='float64')
labels = np.array(labels, dtype ='float64')

In [49]:
history = model.fit(input_sequences, labels, epochs=10, verbose=1, batch_size=64)

Epoch 1/10
528/528 [==============================] - 82s 155ms/step - loss: 0.2324 - accuracy: 0.9643
Epoch 2/10
528/528 [==============================] - 82s 155ms/step - loss: 0.0197 - accuracy: 0.9983
Epoch 3/10
528/528 [==============================] - 82s 155ms/step - loss: 0.0095 - accuracy: 0.9994
Epoch 4/10
528/528 [==============================] - 82s 155ms/step - loss: 0.0077 - accuracy: 0.9995
Epoch 5/10
528/528 [==============================] - 81s 154ms/step - loss: 0.0060 - accuracy: 0.9996
Epoch 6/10
528/528 [==============================] - 82s 154ms/step - loss: 0.0053 - accuracy: 0.9996
Epoch 7/10
528/528 [==============================] - 81s 154ms/step - loss: 0.0051 - accuracy: 0.9997
Epoch 8/10
528/528 [==============================] - 81s 154ms/step - loss: 0.0045 - accuracy: 0.9997
Epoch 9/10
528/528 [==============================] - 81s 154ms/step - loss: 0.0049 - accuracy: 0.9996
Epoch 10/10
528/528 [==============================] - 81s 154ms/step - l

In [0]:
from keras.models import load_model
import pickle

model.save('my_model_embedding.h5')

In [0]:


mymodel = model




def clean_data(input_val):

    input_val=input_val.replace('\n', '')
    input_val=input_val.replace('%20', ' ')
    input_val=input_val.replace('=', ' = ')
    input_val=input_val.replace('((', ' (( ')
    input_val=input_val.replace('))', ' )) ')
    input_val=input_val.replace('(', ' ( ')
    input_val=input_val.replace(')', ' ) ')
    input_val=input_val.replace('1 ', 'numeric')
    input_val=input_val.replace(' 1', 'numeric')
    input_val=input_val.replace("'1 ", "'numeric ")
    input_val=input_val.replace(" 1'", " numeric'")
    input_val=input_val.replace('1,', 'numeric,')
    input_val=input_val.replace(" 2 ", " numeric ")
    input_val=input_val.replace(' 3 ', ' numeric ')
    input_val=input_val.replace(' 3--', ' numeric--')
    input_val=input_val.replace(" 4 ", ' numeric ')
    input_val=input_val.replace(" 5 ", ' numeric ')
    input_val=input_val.replace(' 6 ', ' numeric ')
    input_val=input_val.replace(" 7 ", ' numeric ')
    input_val=input_val.replace(" 8 ", ' numeric ')
    input_val=input_val.replace('1234', ' numeric ')
    input_val=input_val.replace("22", ' numeric ')
    input_val=input_val.replace(" 8 ", ' numeric ')
    input_val=input_val.replace(" 200 ", ' numeric ')
    input_val=input_val.replace("23 ", ' numeric ')
    input_val=input_val.replace('"1', '"numeric')
    input_val=input_val.replace('1"', '"numeric')
    input_val=input_val.replace("7659", 'numeric')
    input_val=input_val.replace(" 37 ", ' numeric ')
    input_val=input_val.replace(" 45 ", ' numeric ')

    return input_val








def predict_sqli_attack():
    
    repeat=True
    
    beautify=''
    for i in range(20):
        beautify+= "="

    print(beautify) 
    input_val=input("Give me some data to work on : ")
    print(beautify)

    
    if input_val== '0':
        repeat=False
    
    

    input_val=clean_data(input_val)
    input_val=[input_val]



    input_val=myvectorizer.transform(input_val).toarray()
    
    input_val.shape=(1,64,64,1)

    result=mymodel.predict(input_val)


    print(beautify)
    
    
    if repeat == True:
        
        if result>0.5:
            print("ALERT :::: This can be SQL injection")


        elif result<=0.5:
            print("It seems to be safe")
            
        print(beautify)
            
        predict_sqli_attack()
            
    elif repeat == False:
        print( " Good Bye ")

In [0]:
predict_sqli_attack()